In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install datasets

In [25]:
import os
import torch
import pandas as pd
import numpy as np
import transformers
import collections
from transformers import AutoModelForQuestionAnswering, AutoTokenizer,TrainingArguments, Trainer,default_data_collator
from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from datasets import Dataset

In [3]:
data =  pd.read_csv("/content/drive/MyDrive/Hindi-Tamil_Question_Answering_System/data.csv")
data.head()

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil


In [7]:
data.language.value_counts()

hindi    746
tamil    368
Name: language, dtype: int64

## Problem Statement

From the above cells, we can get some insights about the data.
Each entry of dataset contains context-paragraph from where we have to extract the answer, question, answer_text, answer_start, language.
This kind of problem where we extract answer for the question from the given context is called **Extractive Question Answering.**

### Size and Distribution of data:

The dataset contains **1114** examples(**33**% tamil and **67**% hindi) which is very less compared to the other datasets.

One of the famous dataset for extractive question answering is **SQuAD dataset**. The SQUAD has over **23k** paragraphs with **0.1M** questions. But this dataset only contains english examples.

The **XQuAD** dataset made by translating some of the SQuAD example into 11 different languages contains **1190** question answer pairs for all 11 languages including **Hindi** but not **Tamil**.




### Dataset Split for training and testing

Let's randomly shuffle the data and split the data into train and test in the ratio of 85:15

In [10]:
# # Randomize the dataset
# data = data.sample(frac=1, random_state=1).reset_index(drop=True)

# # Calculate index for split 80:20 ratio


# # Split into training and test sets
# training_set = data[:1000].reset_index(drop=True)
# test_set = data[1000:].reset_index(drop=True)

# print(training_set.shape)
# print(test_set.shape)

# training_set.to_csv('/content/drive/MyDrive/Hindi-Tamil_Question_Answering_System/train.csv', index = False)
# test_set.to_csv('/content/drive/MyDrive/Hindi-Tamil_Question_Answering_System/test.csv', index = False)

In [ ]:
## Looking into a tamil example of the data
(data.loc[data["language"]=="tamil"]).head(1)

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil


In [7]:
train = pd.read_csv('/content/drive/MyDrive/Hindi-Tamil_Question_Answering_System/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Hindi-Tamil_Question_Answering_System/test.csv')

In [8]:
# Some of the questions have lot of white space on the left and so lets take of this
# in the preprocess pipeline
train["question"] 

0               இந்திய நாட்டில் எத்தனை மாநிலங்கள் உள்ளன?
1                       मऊ शहर किस नदी के किनारे बसा है?
2                 சிலம்பம் எந்த நாட்டின் தற்காப்புக்கலை?
3          मलेरिया संक्रमण का इलाज किस दवा से किया जाता?
4                           ईद किस धर्म के लोग मनाते है?
                             ...                        
995    कृष्णम राजू ने सबसे पहले किस तेलुगु फिल्म में ...
996                       இசுரேல் நாட்டின் தலைநகரம் எது?
997                      बुर्ज खलीफा की लम्बाई कितनी है?
998                              महाभारत के लेखक कौन थे?
999                     आर्किमिडीज़ का पूरा नाम क्या है?
Name: question, Length: 1000, dtype: object

## Model

Let's try with XLM-RoBERTa Model pretrained on squad dataset 

In [9]:
# The tokenizer for xlm roberta-large pretrained on squad 2 is imported here.
tokenizer = AutoTokenizer.from_pretrained("deepset/xlm-roberta-large-squad2")

In [10]:
pad_on_right = tokenizer.padding_side == "right"
max_length = 384
doc_stride = 128

In [11]:
def prepare_train_features(examples):
    
    #Remove whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

    # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1
# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [12]:
def convert_answers(r):
    start = r[0]
    text = r[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

In [13]:
train = train.sample(frac=1, random_state=42)
train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

In [14]:
df_train = train[:-64].reset_index(drop=True)
df_valid = train[-64:].reset_index(drop=True)

In [15]:
train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

In [16]:
tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
model = AutoModelForQuestionAnswering.from_pretrained("deepset/xlm-roberta-large-squad2")

In [18]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [19]:
args = TrainingArguments(
    f"chaii-qa",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [20]:
data_collator = default_data_collator

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [22]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12433
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 388


Epoch,Training Loss,Validation Loss
0,No log,0.207252


***** Running Evaluation *****
  Num examples = 877
  Batch size = 4
Saving model checkpoint to chaii-qa/checkpoint-388
Configuration saved in chaii-qa/checkpoint-388/config.json
Model weights saved in chaii-qa/checkpoint-388/pytorch_model.bin
tokenizer config file saved in chaii-qa/checkpoint-388/tokenizer_config.json
Special tokens file saved in chaii-qa/checkpoint-388/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=388, training_loss=0.2733117133071742, metrics={'train_runtime': 1686.5002, 'train_samples_per_second': 7.372, 'train_steps_per_second': 0.23, 'total_flos': 8648119510695936.0, 'train_loss': 0.2733117133071742, 'epoch': 1.0})

In [ ]:
trainer.save_model("'/content/drive/MyDrive/Hindi-Tamil_Question_Answering_System/chaii-bert-trained")

In [26]:
def prepare_test_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
        )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [29]:
test['answers'] = test[['answer_start', 'answer_text']].apply(convert_answers, axis=1)
test_dataset = Dataset.from_pandas(test)

In [30]:
test_features = test_dataset.map(
    prepare_test_features,
    batched=True,
    remove_columns=test_dataset.column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [31]:
test_f = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
test_f

  0%|          | 0/1431 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1431
})

In [32]:
raw_predictions = trainer.predict(test_f)

***** Running Prediction *****
  Num examples = 1431
  Batch size = 4


In [33]:
max_answer_length = 30

In [34]:
examples = test_dataset
features = test_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [35]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score
            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [36]:
final_predictions = postprocess_qa_predictions(test_dataset, test_features, raw_predictions.predictions)

Post-processing 114 example predictions split into 1431 features.


  0%|          | 0/114 [00:00<?, ?it/s]

In [39]:
references = [{"id": ex["id"], "answer": ex["answers"]['text'][0]} for ex in test_dataset]

In [40]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [41]:
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res

,id,answer,prediction,jaccard
0,1eacbc70f,பிரித்தானிய,181,0.000
1,dab6fa1cf,C2H5-OH,C2H6O,0.000
2,2da6238ff,यशवंतराव होलकर द्वितीय,तुकोजीराव होलकर द्वितीय,0.500
3,4ab83393f,உருகுவே,பெல்ஜியம்,0.000
4,ac0401666,नई दिल्ली,भारत में नई दिल्ली,0.500
...,...,...,...,...
109,6d9a3f816,वान्देक्स,वान्देक्स,1.000
110,ba9560ca9,८.३ मिनट,८.३ मिनट,1.000
111,cc4c69225,सन ३३० ईसापूर्व,ईसापूर्व 330,0.250
112,de2d1e19a,கிமு 356 ஆம் ஆண்டு சூலை மாதம் 6,"சூலை 20, கிமு 356",0.375


In [42]:
print("The average Jaccard Score is ",res.jaccard.mean())

The average Jaccard Score is  0.5863913255360624
